A function that depends on an object and implementation of specific methods...

In [1]:
class Lamp:
    def turn_on(self):
        print("Lamp is on")
        
    def turn_off(self):
        print("Lamp is off")

def operate(lamp):
    lamp.turn_on()
    lamp.turn_off()

lamp = Lamp()
operate(lamp)

Lamp is on
Lamp is off


An attempt to extend usage to other object types...

In [2]:
class Fan:
    def turn_on(self):
        print("Fan is on")
        
    def turn_off(self):
        print("Fan is off")

class Lamp:
    def turn_on(self):
        print("Lamp is on")
        
    def turn_off(self):
        print("Lamp is off")

def operate(lamp):
    lamp.turn_on()
    lamp.turn_off()

lamp = Lamp()
fan = Fan()
operate(lamp)

# !!! Fan is not a Lamp
operate(fan)

Lamp is on
Lamp is off
Fan is on
Fan is off


Use Dependency Inversion Principle (DIP) to introduce an abstraction and `operate` will work with that instead.

Now, `operate` is a function that accepts an abstract `Switchable` object.

In [3]:
from abc import ABC, abstractmethod


class Switchable(ABC):
    @abstractmethod
    def turn_on(self):
        pass

    @abstractmethod
    def turn_off(self):
        pass


class Fan(Switchable):
    def turn_on(self):
        print("Fan is on")

    def turn_off(self):
        print("Fan is off")


class Lamp(Switchable):
    def turn_on(self):
        print("Lamp is on")

    def turn_off(self):
        print("Lamp is off")


def operate(device: Switchable):
    device.turn_on()
    device.turn_off()


lamp = Lamp()
fan = Fan()
operate(lamp)
operate(fan)

Lamp is on
Lamp is off
Fan is on
Fan is off


Next, instead of an `operate` function, we start with a `DeviceOperator` class that instantiates a `Lamp`. 
- Since we initialize `Lamp` inside the DeviceOperator class, we cannot easily change the device to Fan or any other device without changing the DeviceOperator class.
- This pattern of initializing other classes can happen during initialization of `DeviceOperator` or in subsequent methods.

In [4]:
class Lamp:
    def turn_on(self):
        print("Lamp is on")

    def turn_off(self):
        print("Lamp is off")

class DeviceOperator:
    def __init__(self):
        self.device = Lamp()

    def run(self):
        self.device.turn_on()
        self.device.turn_off()


lamp = Lamp()
lamp_operator = DeviceOperator()
lamp_operator.run()

# can't use DeviceOperator for Fan since it is tightly coupled to Lamp
fan = Fan()


Lamp is on
Lamp is off


Use Dependency Injection (DI). Separate creation (of Lamp, Fan, other device) from usage. 
- We achieve this by create our `Lamp` object outside of our initialization of `DeviceOperator`. `DeviceOperator` takes a `Lamp` as a dependency. 
- In fact, we can pass in other objects like mocks for testing or a `Fan`.

In [5]:
class Lamp:
    def turn_on(self):
        print("Lamp is on")

    def turn_off(self):
        print("Lamp is off")

class DeviceOperator:
    def __init__(self, device):
        self.device = device

    def run(self):
        self.device.turn_on()
        self.device.turn_off()


lamp = Lamp()
lamp_operator = DeviceOperator(device=lamp)
lamp_operator.run()

fan = Fan()
fan_operator = DeviceOperator(device=fan)
fan_operator.run()


Lamp is on
Lamp is off
Fan is on
Fan is off


Using Dependency Injection (DI) to achieve Dependency Inversion Principle (DIP) with abstract interface layer.

In [6]:
from abc import ABC, abstractmethod


class Switchable(ABC):
    @abstractmethod
    def turn_on(self):
        pass

    @abstractmethod
    def turn_off(self):
        pass


class Fan(Switchable):
    def turn_on(self):
        print("Fan is on")

    def turn_off(self):
        print("Fan is off")


class Lamp(Switchable):
    def turn_on(self):
        print("Lamp is on")

    def turn_off(self):
        print("Lamp is off")

class TestDevice(Switchable):
    def turn_on(self):
        print("TestDevice is on")

    def turn_off(self):
        print("TestDevice is off")

class DeviceOperator:
    def __init__(self, device: Switchable):
        self.device = device

    def run(self):
        self.device.turn_on()
        self.device.turn_off()


lamp = Lamp()
lamp_operator = DeviceOperator(device=lamp)
lamp_operator.run()

fan = Fan()
fan_operator = DeviceOperator(device=fan)
fan_operator.run()

test_operator = DeviceOperator(device=TestDevice())
test_operator.run()


class Foo:
    pass


Lamp is on
Lamp is off
Fan is on
Fan is off
TestDevice is on
TestDevice is off


How to remember the difference between DI and DIP?

- Dependency injection - focus is separating creation from use
- Dependency inversion - focus is on abstraction and interactions. Interaction layer is more flexible. Goes a step further than DI because interactions are interchangeable in addition to components themselves.

In [7]:
from abc import ABC, abstractmethod


# Existing abstraction
class Switchable(ABC):
    @abstractmethod
    def turn_on(self):
        pass

    @abstractmethod
    def turn_off(self):
        pass


# New abstraction for adjustable devices
class Adjustable(ABC):
    @abstractmethod
    def adjust(self, setting):
        pass


# Implementing a new device with both behaviors
class DimmableLamp(Switchable, Adjustable):
    def turn_on(self):
        print("Dimmable Lamp turned on.")

    def turn_off(self):
        print("Dimmable Lamp turned off.")

    def adjust(self, setting):
        print(f"Dimmable Lamp adjusted to setting {setting}.")


# Modified DeviceOperator to optionally handle adjustable devices
class DeviceOperator:
    def __init__(self, device: Switchable):
        self.device = device

    def operate(self):
        self.device.turn_on()
        # ... some operations
        self.device.turn_off()

    def adjust_device(self, setting):
        if isinstance(self.device, Adjustable):
            self.device.adjust(setting)


# Usage
dimmable_lamp = DimmableLamp()
dimmable_lamp_operator = DeviceOperator(dimmable_lamp)
dimmable_lamp_operator.operate()  # Standard operation for all devices
dimmable_lamp_operator.adjust_device(5)  # New operation for adjustable devices

fan = Fan()
fan_operator = DeviceOperator(fan)
fan_operator.operate()
fan_operator.adjust_device(5)  # No operation performed.

Dimmable Lamp turned on.
Dimmable Lamp turned off.
Dimmable Lamp adjusted to setting 5.
Fan is on
Fan is off
